In [ ]:
# !pip3 install gym gym-retro

# !pip uninstall gym
# !pip3 install gym==0.21.0

In [1]:
import retro
import time

In [ ]:
# See the different retro games
# retro.data.list_games()

In [3]:
# Starts up the game environment
#https://wowroms.com/en/roms/list?search=street+fighter+special+champion+edition+usa
    # python -m retro.import
env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')

In [ ]:
#env.close()

In [ ]:
# env.observation_space

In [ ]:
# env.observation_space.sample()

In [ ]:
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        time.sleep(0.01)
        if reward > 0:
            print(reward)

500.0
500.0
1000.0
500.0
300.0
500.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
1000.0
10000.0
10000.0
400.0
1000.0
500.0
100.0
500.0
100.0
1500.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
10000.0
500.0
300.0
100.0


In [ ]:
info

In [ ]:
# env.close()

# Setup Environment

In [2]:
#Observation preprocess
#preprocess---grayscale,frame delta,resize the frame so that we have less pixels
# filter the action-parameter

from gym import Env
from gym.spaces import MultiBinary, Box
import numpy as np
import cv2
import time
from matplotlib import pyplot as plt

In [3]:
class StreetFighter(Env): 
    def __init__(self):
        super().__init__()
        # Specify action space and observation space 
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        # Startup and instance of the game 
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions=retro.Actions.FILTERED)
    
    def reset(self):
        # Return the first frame 
        obs = self.game.reset()
        obs = self.preprocess(obs) 
        self.previous_frame = obs 
        
        # Create a attribute to hold the score delta 
        self.score = 0 
        return obs
    
    def preprocess(self, observation): 
        # Grayscaling 
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Resize 
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        # Add the channels value
        channels = np.reshape(resize, (84,84,1))
        return channels 
    
    def step(self, action): 
        # Take a step 
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs) 
        
        # Frame delta 
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs 
        
        # Reshape the reward function
        reward = info['score'] - self.score 
        self.score = info['score'] 
        
        return frame_delta, reward, done, info
    
    def render(self, *args, **kwargs):
        self.game.render()
        
    def close(self):
        self.game.close()

In [ ]:
env = StreetFighter()

In [ ]:
env.observation_space.shape

In [ ]:
env.action_space.shape

In [ ]:
# Reset game to starting state
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        time.sleep(0.01)
        if reward > 0: 
            print(reward)

In [ ]:
obs = env.reset()

In [ ]:
obs, reward, done, info = env.step(env.action_space.sample())

In [ ]:
plt.imshow(cv2.cvtColor(obs, cv2.COLOR_BGR2RGB))

In [ ]:
# !pip install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio===0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [ ]:
# !pip install stable-baselines3[extra]

In [ ]:
# !pip install optuna

In [4]:
# Importing the optimzation frame - HPO
import optuna
# PPO algo for RL
from stable_baselines3 import PPO
# Bring in the eval policy method for metric calculation
from stable_baselines3.common.evaluation import evaluate_policy
# Import the sb3 monitor for logging 
from stable_baselines3.common.monitor import Monitor
# Import the vec wrappers to vectorize and frame stack
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
# Import os to deal with filepaths
import os

In [5]:
LOG_DIR = './logs/'
OPT_DIR = './opt/'

In [6]:
# Function to return test hyperparameters - define the object function
def optimize_ppo(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    }

In [ ]:
# SAVE_PATH = os.path.join('trial_{}_best_model'.format(1))

In [ ]:
# # Run a training loop and return mean reward 
# def optimize_agent(trial):
#     try:
#         model_params = optimize_ppo(trial) 

#         # Create environment 
#         env = StreetFighter()
#         env = Monitor(env, LOG_DIR)
#         env = DummyVecEnv([lambda: env])
#         env = VecFrameStack(env, 4, channels_order='last')

#         # Create algo 
#         model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
#         model.learn(total_timesteps=30000)
#         #model.learn(total_timesteps=100000)

#         # Evaluate model 
#         mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=)
#         env.close()

#         SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
#         model.save(SAVE_PATH)

#         return mean_reward

#     except Exception as e:
#         return -1000

In [7]:
# Run a training loop and return mean reward 
def optimize_agent(trial):
    model_params = optimize_ppo(trial) 

    # Create environment 
    env = StreetFighter()
    env = Monitor(env, LOG_DIR)
    env = DummyVecEnv([lambda: env])
    env = VecFrameStack(env, 4, channels_order='last')
    # Create algo 
    model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
    model.learn(total_timesteps=30000)    #model.learn(total_timesteps=100000)

    # Evaluate model 
    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
    env.close()

    SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
    model.save(SAVE_PATH)

    return mean_reward


In [8]:
# Creating the experiment 
study = optuna.create_study(direction='maximize')
study.optimize(optimize_agent, n_trials=10, n_jobs=1)
# study.optimize(optimize_agent, n_trials=100, n_jobs=1)

[I 2022-08-17 12:51:34,774] A new study created in memory with name: no-name-99bc8821-cd20-4a01-8658-f3cd6d7097bf
C:\Users\panch\AppData\Roaming\Python\Python38\site-packages\stable_baselines3\ppo\ppo.py:146: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 6461`, after every 100 untruncated mini-batches, there will be a truncated mini-batch of size 61
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=6461 and n_envs=1)
  warnings.warn(
[I 2022-08-17 13:03:26,673] Trial 0 finished with value: 3300.0 and parameters: {'n_steps': 6461, 'gamma': 0.801677180763084, 'learning_rate': 6.997691571445032e-05, 'clip_range': 0.23970394480575144, 'gae_lambda': 0.8915495127522939}. Best is trial 0 with value: 3300.0.
C:\Users\panch\AppData\Roaming\Python\Python38\site-packages\stable_baselines3\ppo\ppo.py:146: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutB

In [9]:
study.best_params

{'n_steps': 5434,
 'gamma': 0.8493518570779583,
 'learning_rate': 3.500963311405907e-05,
 'clip_range': 0.38894254030156405,
 'gae_lambda': 0.976061027822489}

In [10]:
study.best_trial

FrozenTrial(number=8, values=[6700.0], datetime_start=datetime.datetime(2022, 8, 17, 13, 59, 37, 645486), datetime_complete=datetime.datetime(2022, 8, 17, 14, 6, 25, 727865), params={'n_steps': 5434, 'gamma': 0.8493518570779583, 'learning_rate': 3.500963311405907e-05, 'clip_range': 0.38894254030156405, 'gae_lambda': 0.976061027822489}, distributions={'n_steps': IntUniformDistribution(high=8192, low=2048, step=1), 'gamma': LogUniformDistribution(high=0.9999, low=0.8), 'learning_rate': LogUniformDistribution(high=0.0001, low=1e-05), 'clip_range': UniformDistribution(high=0.4, low=0.1), 'gae_lambda': UniformDistribution(high=0.99, low=0.8)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=8, state=TrialState.COMPLETE, value=None)

In [12]:
model = PPO.load(os.path.join(OPT_DIR, 'trial_8_best_model.zip'))

# Setup Callback

In [13]:
# Import base callback 
from stable_baselines3.common.callbacks import BaseCallback

In [14]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [15]:
CHECKPOINT_DIR = './train/'

In [16]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

# Train Model

In [6]:
# Create environment 
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [18]:
model_params = study.best_params
model_params['n_steps'] = 7488  # set n_steps to 7488 or a factor of 64
# model_params['learning_rate'] = 5e-7
model_params

{'n_steps': 7488,
 'gamma': 0.8493518570779583,
 'learning_rate': 3.500963311405907e-05,
 'clip_range': 0.38894254030156405,
 'gae_lambda': 0.976061027822489}

In [19]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [20]:
# Reload previous weights from HPO
model.load(os.path.join(OPT_DIR, 'trial_8_best_model.zip'))

In [21]:
# Kick off training 
model.learn(total_timesteps=100000, callback=callback)
# model.learn(total_timestep=5000000) 

Logging to ./logs/PPO_12
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6.62e+03 |
|    ep_rew_mean     | 2.3e+03  |
| time/              |          |
|    fps             | 601      |
|    iterations      | 1        |
|    time_elapsed    | 12       |
|    total_timesteps | 7488     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.62e+03    |
|    ep_rew_mean          | 2.3e+03     |
| time/                   |             |
|    fps                  | 176         |
|    iterations           | 2           |
|    time_elapsed         | 85          |
|    total_timesteps      | 14976       |
| train/                  |             |
|    approx_kl            | 0.017377233 |
|    clip_fraction        | 0.0407      |
|    clip_range           | 0.389       |
|    entropy_loss         | -8.3        |
|    explained_variance   | 5.02e-05    |
|    

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 8.59e+03   |
|    ep_rew_mean          | 2.06e+04   |
| time/                   |            |
|    fps                  | 101        |
|    iterations           | 12         |
|    time_elapsed         | 888        |
|    total_timesteps      | 89856      |
| train/                  |            |
|    approx_kl            | 0.07164568 |
|    clip_fraction        | 0.172      |
|    clip_range           | 0.389      |
|    entropy_loss         | -4.33      |
|    explained_variance   | 0.0122     |
|    learning_rate        | 3.5e-05    |
|    loss                 | 1.72       |
|    n_updates            | 110        |
|    policy_gradient_loss | -0.00255   |
|    value_loss           | 592        |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 8.4e+03    |
|    ep_rew_mean

# Evaluate the Model

In [7]:
model = PPO.load('./opt/trial_8_best_model.zip')

In [8]:
mean_reward, _ = evaluate_policy(model, env, render=True, n_eval_episodes=1)

C:\Users\panch\anaconda3\lib\site-packages\pyglet\image\codecs\wic.py:406: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


In [9]:
mean_reward

6700.0

# Test out the Model

In [10]:
obs = env.reset()

In [11]:
obs.shape

(1, 84, 84, 4)

In [12]:
env.step(model.predict(obs)[0])

(array([[[[  0,   0,  36,   0],
          [  0,   0,  36,   0],
          [  0,   0,  33,   0],
          ...,
          [  0,   0,  37,   0],
          [  0,   0,  34,   0],
          [  0,   0,  36,   0]],
 
         [[  0,   0,  36,   0],
          [  0,   0,  36,   0],
          [  0,   0,  35,   0],
          ...,
          [  0,   0,  36,   0],
          [  0,   0,  40,   0],
          [  0,   0,  40,   0]],
 
         [[  0,   0,  36,   0],
          [  0,   0,  36,   0],
          [  0,   0,  36,   0],
          ...,
          [  0,   0,  36,   0],
          [  0,   0,  36,   0],
          [  0,   0,  36,   0]],
 
         ...,
 
         [[  0,   0, 162,   0],
          [  0,   0, 159,   0],
          [  0,   0, 159,   0],
          ...,
          [  0,   0, 159,   0],
          [  0,   0, 159,   0],
          [  0,   0, 159,   0]],
 
         [[  0,   0, 162,   0],
          [  0,   0, 162,   0],
          [  0,   0, 162,   0],
          ...,
          [  0,   0, 162,   0],
 

In [13]:
# Reset game to starting state
obs = env.reset()
# Set flag to flase
done = False

for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        action = model.predict(obs)[0]
        obs, reward, done, info = env.step(action)
        time.sleep(0.01)
        if reward > 0:
            print(reward)

[400.]
[500.]
[1000.]
[300.]
[1000.]
[500.]
[1500.]
[100.]
[100.]
[100.]
[100.]
[100.]
[100.]
[100.]
[100.]
[100.]
[100.]
[100.]
[100.]
[100.]
[100.]
[100.]
[100.]
[1000.]
[1000.]
[1000.]
[1000.]
[1000.]
[1000.]
[1000.]
[500.]
[500.]
[100.]
[500.]
[1000.]
[1500.]
[100.]
[1000.]
[1000.]
[1000.]
[1000.]
[1000.]
[1000.]
[1000.]
[400.]
[100.]
[500.]
[1000.]
[300.]
[500.]


In [14]:
info

[{'enemy_matches_won': 2,
  'score': 27800,
  'matches_won': 0,
  'continuetimer': 10,
  'enemy_health': 0,
  'health': 0,
  'episode': {'r': 27800, 'l': 10848, 't': 518.158784},
  'terminal_observation': array([[[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          ...,
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],
  
         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          ...,
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],
  
         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          ...,
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          ...,
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],
  
         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          ...,
          [0, 0, 0, 0],
          [0, 0, 